In [ ]:
# pip install librosa
# pip install keras
# pip install tensorflow
# pip install os

In [2]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


import module.lstm as LSTM_MODEL
import module.preprocess as PROCESSING

In [3]:
encoder = LabelEncoder()
FEATURES = "MFCC20"
ROOT = "C:/Users/SSAFY/Desktop/S08P22A603/AI"
SR = 16000
TARGET_EPOCH = 50
PLUS_EPOCH = 50


TRAIN_PATH = f"{ROOT}/train"
VAL_PATH = f"{ROOT}/val"
TENSER_PATH = f"{ROOT}/tensor/{FEATURES}"
CHECKPOINT_PATH = f"{ROOT}/checkpoint"
CHECK_POINT_FILE = f"{FEATURES}/checkpoint{TARGET_EPOCH}.h5"
REPLACE_FILE = f"{FEATURES}/checkpoint{(TARGET_EPOCH+PLUS_EPOCH)}.h5"

In [4]:
# train_X, train_labels = PROCESSING.createDATA(TRAIN_PATH,SR)
# train_X, train_labels = np.load(f'{TENSER_PATH}/train_X.npy'), np.load(f'{TENSER_PATH}/train_labels.npy')

In [5]:
#val_X, val_labels = PROCESSING.createDATA(VAL_PATH,SR)
#val_X, val_labels = np.load(f'{TENSER_PATH}/val_X.npy'), np.load(f'{TENSER_PATH}/val_labels.npy')

In [ ]:
X, Y = PROCESSING.createDATA(TRAIN_PATH,SR)
np.save(f'{TENSER_PATH}/train_X.npy', X)
np.save(f'{TENSER_PATH}/train_X.npy', Y)

In [7]:
# np.save(f'{TENSER_PATH}/train_X.npy', train_X)
# np.save(f'{TENSER_PATH}/train_labels.npy', train_labels)
# np.save(f'{TENSER_PATH}/val_X.npy', val_X)
# np.save(f'{TENSER_PATH}/val_labels.npy', val_labels)
# np.save(f'{TENSER_PATH}/label.npy',np.concatenate([train_labels, val_labels]))

In [8]:
encoder.fit(np.concatenate([train_labels, val_labels]))
train_y = encoder.transform(train_labels)
val_y = encoder.transform(val_labels)

In [ ]:
np.save(f'{TENSER_PATH}/label.npy',np.concatenate([train_labels, val_labels]))

In [9]:
model = LSTM_MODEL.create_lstm_model(len(encoder.classes_),sr = SR)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_callback = LSTM_MODEL.Checkpoint(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


if os.path.isfile(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}"):
    model.load_weights(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


In [ ]:
model.fit(train_X, to_categorical(train_y,num_classes=len(encoder.classes_)), epochs= (TARGET_EPOCH+PLUS_EPOCH), validation_data=(val_X, to_categorical(val_y,num_classes=len(encoder.classes_))), callbacks=[checkpoint_callback], initial_epoch=TARGET_EPOCH)
#model.save(f"{CHECKPOINT_PATH}/{REPLACE_FILE}")

In [ ]:
test_wav = PROCESSING.load_wav_file(f"{ROOT}/sample/윤하.wav")
test_wav = PROCESSING.preprocess_features([test_wav])

In [ ]:
pred = model.predict(test_wav)

In [ ]:
pred

In [ ]:
pred = model.predict(test_wav)
top_5 = np.argsort(pred, axis=1)[:, -5:]
top_5_labels = encoder.inverse_transform(top_5.T.ravel())

In [ ]:
top_5_labels

In [ ]:
pred